<a href="https://colab.research.google.com/github/k-washi/python-cookbook/blob/master/ml/2/cart_pole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## モデルフリー

エージェントは現在の観測結果を取得し、何らかの計算を行い、その結果、実行すべきアクションを取得できる。
複雑な環境におけるモデル構築が容易。

## モデルベース

次の観測および報酬を予測することで、エージェントは最善のアクションを選択しようとする。
ボードゲームなどの決定論的な環境

---------

## ポリシーベース

エージェントのポリシー(アクションの確率分布)を直接近似

## 値ベース

ポリシーを考えうるすべてのアクションに関して計算

---

## 



In [1]:
!pip install tensorboardX==1.9

In [0]:
import gym
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim


HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [0]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )
    def forward(self, x):
        return self.net(x)

In [4]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])
print(Episode)
print(EpisodeStep)

<class '__main__.Episode'>
<class '__main__.EpisodeStep'>


In [0]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        step = EpisodeStep(observation=obs, action=action)
        episode_steps.append(step)
        if is_done:
            e = Episode(reward=episode_reward, steps=episode_steps)
            batch.append(e)
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

In [0]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for reward, steps in batch:
        if reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, steps))
        train_act.extend(map(lambda step: step.action, steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [7]:
env = gym.make("CartPole-v0")
    
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n
net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
writer = SummaryWriter(comment="-cartpole")

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [9]:
for iter_no, batch in enumerate(iterate_batches(env, net,BATCH_SIZE)):
        obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()
        print("%d: loss=%.3f, reward_mean=%.1f, rw_bound=%.1f" % (
            iter_no, loss_v.item(), reward_m, reward_b))
        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_bound", reward_b, iter_no)
        writer.add_scalar("reward_mean", reward_m, iter_no)
        if reward_m > 199:
            print("Solved!")
            break
writer.close()

0: loss=0.685, reward_mean=20.0, rw_bound=21.0
1: loss=0.666, reward_mean=31.4, rw_bound=33.0
2: loss=0.665, reward_mean=29.2, rw_bound=34.5
3: loss=0.635, reward_mean=37.3, rw_bound=39.5
4: loss=0.635, reward_mean=38.5, rw_bound=47.5
5: loss=0.616, reward_mean=39.7, rw_bound=42.0
6: loss=0.622, reward_mean=46.2, rw_bound=53.0
7: loss=0.614, reward_mean=53.8, rw_bound=60.5
8: loss=0.611, reward_mean=52.1, rw_bound=62.0
9: loss=0.601, reward_mean=56.1, rw_bound=59.5
10: loss=0.597, reward_mean=58.2, rw_bound=61.0
11: loss=0.575, reward_mean=53.5, rw_bound=69.5
12: loss=0.586, reward_mean=69.2, rw_bound=73.5
13: loss=0.570, reward_mean=53.4, rw_bound=60.5
14: loss=0.563, reward_mean=60.4, rw_bound=67.0
15: loss=0.555, reward_mean=64.0, rw_bound=64.0
16: loss=0.584, reward_mean=60.1, rw_bound=55.5
17: loss=0.564, reward_mean=63.5, rw_bound=72.5
18: loss=0.562, reward_mean=56.1, rw_bound=64.0
19: loss=0.550, reward_mean=74.0, rw_bound=82.0
20: loss=0.542, reward_mean=75.8, rw_bound=86.0
21